In [1]:
%pip install scikit-learn


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text
import numpy as np
import sys
sys.path.append('../')
from utils import remove_similar_rows_per_player, find_lines_with_player, del_patterns, map_emoji_to_description, remove_accents
import emoji
from googletrans import Translator



In [3]:
url = 'https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/data_files/all_data_v3.csv'
df = pd.read_csv(url)

# First Transformation (with Pattern deletion)
The first transformation focus on the deletion of Patterns to clean the corpus.

## Filter out German data

In [4]:
# Filter out the German data and reindex
df_ger = df[df["language"] == "de"]
df_ger = df_ger.reset_index(drop=True)

In [5]:
# Remove the similiar rows (The Function is imported from utils on top)
df = remove_similar_rows_per_player(df_ger, df_ger['player'].unique())

## Transform Data to lower case

In [6]:
# transform data into lower case
data_lower = df.copy()

data_lower['data'] = data_lower['data'].str.lower()
data_lower['player'] = data_lower['player'].str.lower()



## Delte Patterns

In [7]:
# delete content patterns
data_wo_pattern = data_lower.copy()
data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: re.sub(r"^{\'content\': \'", "", str(x)))

In [8]:
pattern = ['nutze kicker', 
           'mit dem', 
           'nur €2,49', 
           'bereits pur-abonnent?', 
           'alle antworten', 
           'hinweis zur verarbeitung', 
           'werbung','olympia-verlags', 
           'bild', 'g+j medien gmbh', 
           'services',
           'kurz-link dieses artikels lautet:', 
           'http://epaper.welt.de',
           'stephan von nocks',
           'www.faz.net',
           'mcfit mitgliedschaft',
           'fitx-vertrag',
           'kündigeneasyfitness',
           'proteinbedarf',
           'fitnessland',
           'kündigeneasyfitness',
           'trainingspuls berechnen',
           'pulsrechner',
           'fitseveneleven-mitgliedschaft',
           'alkoholabbau & promille',
           'index.promillerechner',
           'mitgliedschaft kündigen',
           'promillerechner',
           'ihr body-mass-index',
           'bmi rechner',
           'kalorienrechner',
           'grundumsatz & kalorienbedarf',
           'partnerangebote',
           'newsletter',
           'journalismus der presse',
           'abonnieren',
           '(apa)',
           'foto:',
           'quelle:',
           'lesezeit:',
           'lesen sie mehr',
           'stellenmarkt der sz.',
           'bewerben sie sich jetzt',
           'gutscheine',
           'vergleichsportal',
           'stern plus-inhalten',
           'jederzeit kündbar',
           'bereits registriert?',
           'zur startseite',
           'öffnet in neuem tab oder fenster',
           'vollansicht der tabelle unter'        
           ]

In [9]:
data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: del_patterns(str(x), pattern))

## Transform Emojis to text

In [10]:
data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: re.sub(r'[^\w\s]', lambda match: map_emoji_to_description(match.group(), language = 'de',), str(x)))

## Remove noise

In [11]:
data_rm_1 = data_wo_pattern.copy()

# strip_numeric -> removing digits (https://gensimr.news-r.org/reference/strip_numeric.html)
data_rm_1['data'] = data_rm_1['data'].apply(strip_numeric)

# strip_punctutation -> removing punctations (https://gensimr.news-r.org/reference/strip_punctuation.html)
data_rm_1['data'] = data_rm_1['data'].apply(strip_punctuation)

# strip multiple whitespaces also \n -> (https://radimrehurek.com/gensim/parsing/preprocessing.html#gensim.parsing.preprocessing.strip_multiple_whitespaces)
data_rm_1['data'] = data_rm_1['data'].apply(strip_multiple_whitespaces)

#strip spanish accents
data_rm_1['data'] = data_rm_1['data'].apply(lambda x: remove_accents(str(x)))

#strip_short deletes words smaller 3
data_rm_1['data'] = data_rm_1['data'].apply(strip_short)
 
#strip links
data_rm_1['data'] = data_rm_1['data'].apply(lambda x: re.sub(r'http\S+', '', str(x)))

# Remove stopwords

In [12]:
german_stop_words = stopwords.words('german')

# Create a list of stop words to remove 
stop_words_to_remove = ['nicht', 'nichts']

stop_words_to_add = ['vor', 'in', 'den', 'dem', 'beim', 'wir', 'der', 'ist','ende', 'seite', 'ersten', 'fürs', 'eh', 'blick', 'schon', 'zumal', 'erst', 'mal', 'bild', 't', '(dpa)']

# Remove the stop words to remove from the stop words list
for word in stop_words_to_remove:
  german_stop_words.remove(word)

german_stop_words.append(stop_words_to_add)

# Define a function to apply remove_stopwords on a column
def remove_stopwords_from_text(text):
    return remove_stopwords(text, stopwords=german_stop_words)

# Apply the remove_stopwords function to the 'text' column using the apply method
data_rm_1['data'] = data_rm_1['data'].apply(remove_stopwords_from_text)


In [13]:
data_rm_1.reset_index(drop=True, inplace=True)


In [14]:
data_rm_1['data'].iloc[0]

'trainer xabi alonso mitgereisten fans monacomit familie genoss xabi alonso team hotel monaco moment erfolges dramatischen einzug achtelfinale europa league ,,aber gab zeit feiern" sagte trainer bayer leverkusen personlich wohl wichtigste spiel jungen trainerlaufbahn erlebt alonso ,,es intensiv fur ersten mal situation ruhig fokussiert besten entscheidungen treffen bessere emotionale kontrolle besser spiel leider konnten nicht verteidigen zufrieden leistung mannschaft "wie gross anspannung elfmeterschiessen alonso ,,ehrlich vertrauen mittwoch training elfmeter geubt gut azmoun gut amiri gut schon klare idee spieler schiessen konnte bundesliga viele elfer verschossen funfmal folge getroffen effektiv ende passieren spieler gut gemacht gibt trick lotto gestern glucklichen "glucksmoment exequiel palacios hebt tor abnach euro highlight geht bundesliga sonntag uhr beim freiburg ,,wir kennen freiburg gerade guten moment spielen hohen intensitat ahnlich mainz vielleicht spiel mainz gute schule

## Save cleaned data in csv

In [15]:
data_rm_1.to_csv('/Users/kevingiesen/Library/Mobile Documents/com~apple~CloudDocs/BIPM Master/Semester 2/TWSM/TWSM Project/SS23-BIPM-Analytics-Lab---Group-4-repository/Preprocessing/data_clean/de_clean_1.csv', index=False)

# Second Transformation (setence with Playernames)
The second transformation focus on the deletion of sentences to clean the corpus.

## Get lines and following lines where the Player name appears in the corpus 

In [16]:
# delete content pattern
df['data'] = df['data'].apply(lambda x: re.sub(r"^{\'content\': \'", "", str(x)))

In [17]:
# transform data into lower case
data_lower = df.copy()

data_lower['data'] = data_lower['data'].str.lower()
data_lower['player'] = data_lower['player'].str.lower()


In [18]:
data_with_playernames = find_lines_with_player(data_lower, data_lower['player'].unique(), n_lines = 1)

## Transform Emojis

In [19]:
data_with_playernames['data'] = data_with_playernames['data'].apply(lambda x: re.sub(r'[^\w\s]', lambda match: map_emoji_to_description(match.group(), language = 'de',), str(x)))

## Delete unnecessary

In [20]:
data_rm_2 = data_with_playernames.copy()

# strip_numeric -> removing digits (https://gensimr.news-r.org/reference/strip_numeric.html)
data_rm_2['data'] = data_rm_2['data'].apply(strip_numeric)

# strip_punctutation -> removing punctations (https://gensimr.news-r.org/reference/strip_punctuation.html)
data_rm_2['data'] = data_rm_2['data'].apply(strip_punctuation)

# strip multiple whitespaces also \n -> (https://radimrehurek.com/gensim/parsing/preprocessing.html#gensim.parsing.preprocessing.strip_multiple_whitespaces)
data_rm_2['data'] = data_rm_2['data'].apply(strip_multiple_whitespaces)

#strip spanish accents
data_rm_2['data'] = data_rm_2['data'].apply(lambda x: remove_accents(str(x)))

#strip_short deletes words smaller 3
data_rm_2['data'] = data_rm_2['data'].apply(strip_short)
 
#strip links
data_rm_2['data'] = data_rm_2['data'].apply(lambda x: re.sub(r'http\S+', '', str(x)))

## Stopwords

In [21]:
# Apply the remove_stopwords function to the 'text' column using the apply method
data_rm_2['data'] = data_rm_2['data'].apply(remove_stopwords_from_text)


In [26]:
data_rm_2['data'].iloc[2]

'nutze kicker digitalen plattformen gewohnt werbung tracking zustimmung jederzeit fur zukunft widerrufen details werbe analyse trackern findest unserer datenschutzerklarung cookies tracking ende seite bayer donnerstag achtelfinal hinspiel europa league ferencvaros budapest empfangt robert andrich fehlen exequiel palacios sah defensive mittelfeldspieler beim erfolg elfmeterschiessen monaco dritte gelbe karte gesperrt massive schwachung fur werkself doppelsechs andrich palacios zuletzt erfolgsfaktor andrich ausgerechnet ersten kurzen phase saison bayer gleichzeitig spielerisch uberzeugt erfolgreichen fussball spielt zwangspause einlegen wurmt bedingt schon argerlich irgendwann ware sperre wahrscheinlich gekommen daher lieber eventuell viertel halbfinale sorgen erklart jahrige lacheln'

In [23]:
data_rm_2.to_csv('/Users/kevingiesen/Library/Mobile Documents/com~apple~CloudDocs/BIPM Master/Semester 2/TWSM/TWSM Project/SS23-BIPM-Analytics-Lab---Group-4-repository/Preprocessing/data_clean/de_clean_2.csv', index=False)